# Hyperparameter Tuning using HyperDrive

In [ ]:
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment, ScriptRunConfig

from azureml.train.hyperdrive.policy import NoTerminationPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

from azureml.widgets import RunDetails

## Dataset

### Set up experiment

In [ ]:
ws = Workspace.from_config()
experiment_name = "creditcard-experiment"
project_folder = './creditcard-hyperdrive-project'

experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()

### Connect to Compute

In [ ]:
amlcompute_cluster_name = "automl-cls"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_v3", max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

### Get Dataset

In [ ]:
key = "creditcard-dataset"
description = "Credit Card - Dealing from Imbalance Datasets from https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud"

found = False
if key in ws.datasets.keys():
    print("Found existing dataset, use it.")
    found = True
    dataset = ws.datasets[key] # already registered
    
if not found:
    example_data = "https://media.githubusercontent.com/media/satriawadhipurusa/ml-dataset-collection/master/Fraud-Detection/creditcard-fraud.csv" # uploaded to Git for download
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    dataset = dataset.register(workspace=ws, name=key, description=description)

In [ ]:
dataset.to_pandas_dataframe().head()

## Hyperdrive Configuration

We will use Hyperdrive to search the best hyperparameters of the model. The model will be using `SupportVectorClassifier` or SVC, this model excel in separating hyperplanes of different classes, especially in finding anomaly (**fraud**). This is due the `class_weight` parameter of the model that can be set for imbalanced dataset. 

The followings are the hyperparameter of SVC:

* gamma (Kernel Coefficient): 0.01 - 100
* C (regularization): 0.01 - 100
* class weight: `{0: 0.05, 1: 0.95}`, `{0: 0.1, 1: 0.9}`, `{0: 0.25, 1: 0.75}`

These three parameters are essentials in SVC, and both the gamma and C use a very large parameter space (0.01 - 100). Since we also limited in time and budget, we will use `BayesianParameterSampling` to make the search more informed. Using this sampling method, the algorithm will learn from previous runs to narrow the search space on a parameter that will maximize the objective function, which is maximize the primary metric. Since it's using bayesian, `NoTeriminationPolicy` will be used instead.

Finally, we set the primary metric name as **"AUC Weighted"** instead of Accuracy, it is suited for this type of imbalanced dataset. This metric also used in Automated ML previously so we can compare them on the same ground. The other config, we will maximize `max_total_runs` and `max_duration_minutes`, since bayesian sampling usually took a longer than randomized search or grid search.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = NoTerminationPolicy()

#TODO: Create the different params that you will be using during training.
param_sampling = BayesianParameterSampling({
    "gamma": uniform(0.01, 100),
    "C": uniform(0.01, 100),
    "class_weight": choice(
        "{0: 0.05, 1: 0.95}",
        "{0: 0.1, 1: 0.9}",
        "{0: 0.25, 1: 0.75}")
})

#TODO: Create your estimator and hyperdrive config
environment = Environment.from_conda_specification(name="sklearn-env", file_path="conda.yaml")
arguments = [
    "--gamma",
    1.0,
    "--C",
    1.0,
    "--class_weight",
    "{0: 0.05, 1: 0.95}"
]
estimator = ScriptRunConfig(source_directory=".",
                            script="./training/train.py",
                            arguments=arguments,
                            environment=environment,
                            compute_target=compute_target)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name="AUC_Weighted",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    run_config=estimator,
    policy=early_termination_policy,
    max_total_runs=60,
    max_concurrent_runs=2,
    max_duration_minutes=60
)

In [ ]:
#TODO: Submit your experiment
remote_run = experiment.submit(hyperdrive_run_config)

## Run Details

The different runs show that some run will have much higher metric than other runs. It's the bayesian sampling job to find which parameters can produce the best run with highest metrics. We see that the best metric is **0.919** in AUC Weighted with **0.275 gamma**, **44.808 C**, and `class_weight` of `{0: 0.25, 1: 0.75}`. This is smaller than Automated ML, and hence we can decide that we will not deploy this model but deploy the Automated ML one.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

In [ ]:
best_hyperdrive_run = remote_run.get_best_run_by_primary_metric()

print(f"Best HyperDrive Run:\n\n{best_hyperdrive_run}")
print("==============")
namefile = "outputs/model.joblib"
best_hyperdrive_run.download_file(namefile, namefile) # save the best model
best_hyperdrive_model = joblib.load(open(namefile, "rb"))

print(f"Best HyperDrive Model:\n\n{best_hyperdrive_model}")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [ ]:
# Register the model
best_hyperdrive_run.register_model(model_name="credit-fraud-model", model_path="outputs/model.joblib")

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

